In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
steam = pd.read_csv ('../Datasets/function3.csv.gz', compression='gzip', usecols=['user_id','item_id','sentiment_analysis','recommend','app_name'])

FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/function3.csv.gz'

In [15]:
df = steam
df['item_id'].value_counts()

item_id
440       3401
730       2979
4000      1551
570       1402
218620     994
          ... 
296300       1
350600       1
383230       1
261880       1
267750       1
Name: count, Length: 2408, dtype: int64

In [20]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion_juego(df, input_item_id, num_recommendations=5):
    # Filtrar el DataFrame para obtener solo las filas relacionadas con el juego de entrada
    input_item = df[df['item_id'] == input_item_id]

    if input_item.empty:
        return "El ID de producto no se encuentra en el conjunto de datos."

    # Extraer el usuario y sus recomendaciones para ese juego
    user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='recommend', fill_value=0)

    # Calcular la similitud coseno entre el juego de entrada y todos los demás juegos
    similarity_scores = cosine_similarity(user_item_matrix.T)

    # Obtener las puntuaciones de similitud para el juego de entrada
    input_item_scores = similarity_scores[input_item_id]

    # Obtener los índices de los juegos más similares (excluyendo el juego de entrada)
    similar_items_indices = input_item_scores.argsort()[:-1][-num_recommendations:]

    # Obtener los IDs y nombres de los juegos más similares
    similar_items = user_item_matrix.columns[similar_items_indices]

    # Obtener nombres de juegos correspondientes a los IDs
    game_names_df = df[df['item_id'].isin(similar_items)][['item_id', 'app_name']]

    # Fusionar los DataFrames para obtener nombres correspondientes a los IDs
    merged_df = pd.merge(pd.DataFrame(similar_items, columns=['item_id']), game_names_df, on='item_id')

    # Agrupar juegos recomendados por ID
    grouped_recommendations = merged_df.groupby(['item_id', 'app_name']).size().reset_index(name='counts')

    # Imprimir las recomendaciones de juegos agrupadas por ID
    print(f"Juegos recomendados similares al juego con ID: {input_item_id}")
    for idx, (item_id, game_name, count) in enumerate(zip(grouped_recommendations['item_id'], grouped_recommendations['app_name'], grouped_recommendations['counts']), start=1):
        print(f"{idx}. Juego ID: {item_id}, Name: {game_name}")

# Ejemplo de uso:
input_item_id = 570
recomendacion_juego(df, input_item_id)


Juegos recomendados similares al juego con ID: 570
1. Juego ID: 201805, Name: Orcs Must Die! 2 - Family Ties Booster Pack
2. Juego ID: 212800, Name: Super Crate Box
3. Juego ID: 278970, Name: Digger Online
4. Juego ID: 282780, Name: Stick 'Em Up 2: Paper Adventures
5. Juego ID: 315396, Name: The Crew™ Season Pass


In [4]:
steam[steam['item_id'] == 214950]

,app_name,user_id,item_id,recommend,sentiment_analysis
27777,Total War™: ROME II - Emperor Edition,76561198057086148,214950,1,1
27778,Total War™: ROME II - Emperor Edition,76561198026040814,214950,1,0
27779,Total War™: ROME II - Emperor Edition,76561198088408984,214950,1,0
27780,Total War™: ROME II - Emperor Edition,76561198009498722,214950,0,1
27781,Total War™: ROME II - Emperor Edition,76561198066084140,214950,1,2
...,...,...,...,...,...
27859,Total War™: ROME II - Emperor Edition,76561198047679159,214950,1,2
27860,Total War™: ROME II - Emperor Edition,76561198059323447,214950,0,2
27861,Total War™: ROME II - Emperor Edition,jordywt,214950,1,1
27862,Total War™: ROME II - Emperor Edition,76561198073784330,214950,1,2


Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [5]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

def recomendacion_usuario(usuario_id, steam):
    # Filtrar el DataFrame para obtener las entradas del usuario específico
    usuario_df = steam[steam['user_id'] == usuario_id]

    if usuario_df.empty:
        print(f"No se encontraron datos para el usuario con ID {usuario_id}.")
        return

    # Crear una matriz de usuario-juego para el modelo de vecinos más cercanos
    matriz_usuario_juego = pd.pivot_table(steam, values='sentiment_analysis', index='user_id', columns='app_name', fill_value=0)

    # Transponer la matriz para trabajar en el espacio de juegos
    matriz_juego_usuario = matriz_usuario_juego.T

    # Inicializar el modelo de vecinos más cercanos
    modelo_vecinos = NearestNeighbors(metric='cosine', algorithm='brute')
    modelo_vecinos.fit(matriz_juego_usuario)

    # Encontrar vecinos más cercanos al juego específico
    juego_interes = usuario_df['app_name'].iloc[0]
    indice_juego = matriz_juego_usuario.index.get_loc(juego_interes)
    distancias, indices = modelo_vecinos.kneighbors(matriz_juego_usuario.iloc[indice_juego, :].values.reshape(1, -1), n_neighbors=6)

    # Imprimir la lista de juegos recomendados
    print(f"Recomendación para el usuario con ID: {usuario_id}")
    for i, indice_juego_recomendado in enumerate(indices.flatten()[1:]):  # Excluir al propio juego
        juego_recomendado = matriz_juego_usuario.index[indice_juego_recomendado]
        print(f"{i + 1}. {juego_recomendado}")

# Ejemplo de uso
recomendacion_usuario('green290', steam)


c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Recomendación para el usuario con ID: green290
1. Team Fortress 2
2. Counter-Strike: Global Offensive
3. Portal 2
4. Unturned
5. Left 4 Dead 2
